In [1]:
import random
from typing import *
import json
from itertools import chain

from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, IterableDataset
from transformers import AutoTokenizer, AdamW, AutoModel, get_linear_schedule_with_warmup

In [2]:
import sys
sys.path.append('..')

from constant import rel2id, punct_lst, weak_signals, weak_labels
from utils import to_cuda

# Config

In [3]:
class CFG:
    # data_file = '../aug/diag_codt_new/diag_train.conll'
    # data_file = '../aug/codt/codt_train_fixed.conll'
    data_file = '../data_testset/1to800_1108.json'
    plm = 'hfl/chinese-electra-180g-base-discriminator'
    random_seed = 42
    num_epochs = 2
    batch_size = 128
    plm_lr = 3e-6
    head_lr = 5e-6
    weight_decay = 0.01
    dropout = 0.1
    grad_clip = 2
    scheduler = 'linear'
    warmup_ratio = 0.1
    num_early_stop = 3
    max_length = 160
    hidden_size = 400
    num_labels = 35
    print_every = 300
    eval_every = 600
    cuda = True
    fp16 = True
    debug = False

# Seed

In [4]:
def seed_everything(seed=CFG.random_seed):
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False

seed_everything()

# Data

In [5]:
inter_signals = {
                # '但': 36, '但是': 36, 
                '情况':36, '为什么': 36,
                '吗': 37, '?': 37, '什么': 37,
                # '呢': 37, '吧': 37,
                '请': 39, '麻烦':39, '希望': 39, '让': 39, '咨询': 39}   

In [6]:
origin4change = [rel2id[x] for x in ['root', 'dfsubj', 'sasubj']]
origin4change.extend([i for i in range(21, 35)])

signal_dct = {}
for i, signals in enumerate(weak_signals):
    for s in signals:
        signal_dct[s] = weak_labels[i]
print(signal_dct, len(signal_dct.keys()))

with_inter = True
if with_inter:
    signal_dct.update(inter_signals)
    print(signal_dct, len(signal_dct.keys()))

{'说': 21, '表示': 21, '看到': 21, '显示': 21, '知道': 21, '认为': 21, '希望': 21, '指出': 21, '如果': 25, '假如': 25, '的话': 25, '若': 25, '如': 25, '要是': 25, '倘若': 25, '因为': 23, '所以': 23, '导致': 23, '因此': 23, '造成': 23, '由于': 23, '因而': 23, '但是': 26, '可是': 26, '但': 26, '竟': 26, '却': 26, '不过': 26, '居然': 26, '而是': 26, '反而': 26, '以及': 31, '并且': 31, '或者': 31, '对于': 22, '自从': 22, '上次': 22, '明天': 34, '晚上': 34, '到时候': 34, '再': 34, '然后': 34, '接下来': 34, '最后': 34, '随后': 34, '为了': 28, '使': 28, '为 的': 28, '为 了': 28, '通过': 32, '必须': 32, '点击': 32, '对 吗': 33, '是 吗': 33, '对 吧': 33, '是 吧': 33, '对 ?': 33, '别 的': 24, '另外': 24, '解释': 30, '比如': 30, '例如': 30, '是 这样': 30, '理想': 29, '真 棒': 29, '太 棒': 29, '真差': 29, '太 差': 29, '不 行': 29, '扯皮': 29, '这么 麻烦': 29} 71
{'说': 21, '表示': 21, '看到': 21, '显示': 21, '知道': 21, '认为': 21, '希望': 39, '指出': 21, '如果': 25, '假如': 25, '的话': 25, '若': 25, '如': 25, '要是': 25, '倘若': 25, '因为': 23, '所以': 23, '导致': 23, '因此': 23, '造成': 23, '由于': 23, '因而': 23, '但是': 26, '可是': 26, '但': 26, '竟': 26, '却': 26, '不过': 26, 

In [7]:
class Dependency():
    def __init__(self, idx, word, head, rel):
        self.id = idx
        self.word = word
        self.head = head
        self.rel = rel

    def __str__(self):
        # example:  1	上海	_	NR	NR	_	2	nn	_	_
        values = [str(self.idx), self.word, "_", "_", "_", "_", str(self.head), self.rel, "_", "_"]
        return '\t'.join(values)

    def __repr__(self):
        return f"({self.word}, {self.head}, {self.rel})"

In [8]:
# sample_lst, sample = [], []
# with open(CFG.data_file, 'r', encoding='utf-8') as f:
#     for line in f.readlines():
#         toks = line.strip().split('\t')
#         if len(toks) == 1:
#             sample_lst.append(sample)
#             sample = []
#             continue
        
#         sample.append(Dependency(int(toks[0]), toks[1], toks[6], toks[7]))  # TODO

In [9]:
sample_lst = []
with open(CFG.data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)
for d in data:
    rel_dct = {}
    for tripple in d['relationship']:
        head, rel, tail = tripple
        head_uttr_idx, head_word_idx = [int(x) for x in head.split('-')]
        tail_uttr_idx, tail_word_idx = [int(x) for x in tail.split('-')]
        if head_uttr_idx != tail_uttr_idx:
            continue

        if not rel_dct.get(head_uttr_idx, None):
            rel_dct[head_uttr_idx] = {tail_word_idx: [head_word_idx, rel]}
        else:
            rel_dct[head_uttr_idx][tail_word_idx] = [head_word_idx, rel]
            
    for item in d['dialog']:
        turn = item['turn']
        utterance = item['utterance']
        # dep_lst:List[Dependency] = [Dependency(0, '[root]', -1, '_')]
        sample:List[Dependency] = []

        for word_idx, word in enumerate(utterance.split(' ')):
            head_word_idx, rel = rel_dct[turn].get(word_idx + 1, [word_idx, 'adjct'])  # some word annoted missed, padded with last word and 'adjct'
            sample.append(Dependency(word_idx + 1, word, head_word_idx, rel))  # start from 1

        sample_lst.append(sample)

In [10]:
labels = []
edu_lst = []
for sample in tqdm(sample_lst):
    edu, label = [], []
    for i, dep in enumerate(sample[:-1]):
        word = dep.word
        rel = dep.rel
        
        edu.append(dep)
        
        if f'{word} {sample[i+1].word}' in signal_dct.keys():
            label.append(f'{word}{sample[i+1].word}')
            # if random.random() < 0.9:  # drop
            #     edu.pop(-1)
        elif word in signal_dct.keys():
            label.append(word)
            # if random.random() < 0.9:
            #     edu.pop(-1)
        
        # 'edu'
        if word in punct_lst:
            # if len(label) == 0 and random.random() < 0.5:  # drop
            #     edu, label = [], []
            #     continue
            edu_lst.append(edu)
            labels.append(label)                                
            edu, label = [], []
    
    if len(sample) > 1:
        edu.append(sample[-1])
        if sample[-1].word in signal_dct.keys():
            label.append(word)
    
    # if len(label) == 0 and random.random() < 0.5:
    #     continue
        
    edu_lst.append(edu)
    labels.append(label)

100%|██████████| 20086/20086 [00:00<00:00, 55713.69it/s]


In [11]:
pos_cnt, neg_cnt = 0, 0
for label in labels:
    if len(label) != 0:
        pos_cnt += 1
    else:
        neg_cnt += 1
        
print(pos_cnt, neg_cnt)

8245 20444


In [12]:
tokenizer = AutoTokenizer.from_pretrained(CFG.plm)
print(len(tokenizer))

num_added_toks = tokenizer.add_tokens(['[root]', '[qst]', '[ans]', '[none]'], special_tokens=True)

tokenizer.eos_token = '[EOS]'
print(f'EOS: {tokenizer.eos_token_id}')

tokenizer.root_token = '[root]'
tokenizer.root_token_ids = tokenizer('[root]')['input_ids'][1]
print(f"add token: {tokenizer.root_token} {tokenizer.root_token_ids}")

tokenizer.qst_token = '[qst]'
tokenizer.qst_token_ids = tokenizer('[qst]')['input_ids'][1]
print(f"add token: {tokenizer.qst_token} {tokenizer.qst_token_ids}")

tokenizer.ans_token = '[ans]'
tokenizer.ans_token_ids = tokenizer('[ans]')['input_ids'][1]
print(f"add token: {tokenizer.ans_token} {tokenizer.ans_token_ids}")

tokenizer.none_token = '[none]'
tokenizer.none_token_ids = tokenizer('[none]')['input_ids'][1]
print(f"add token: {tokenizer.none_token} {tokenizer.none_token_ids}")

# tokenizer.signal_token = '[signal]'
# tokenizer.signal_token_ids = tokenizer('[signal]')['input_ids'][1]
# print(f"add token: {tokenizer.signal_token} {tokenizer.signal_token_ids}")

print(len(tokenizer))
CFG.tokenizer = tokenizer

21128
EOS: 100
add token: [root] 21128
add token: [qst] 21129
add token: [ans] 21130
add token: [none] 21131
21132


In [13]:
# class EduDataset(Dataset):
#     def __init__(self, cfg, train, prompt, edu_lst, labels=None):
#         self.train = train
#         self.cfg = cfg
#         self.prompt = prompt
    
#         self.inference = True if labels is None else False
        
#         if self.inference: 
#             self.inputs, self.heads, self.rels, self.masks = self.read_data(edu_lst)
#         else:
#             self.inputs, self.heads, self.rels, self.labels, self.masks = self.read_data(edu_lst, labels)
        
#     def read_data(self, edu_lst, labels=None):
#         inputs, offsets = [], []
#         tags, heads, rels, masks = [], [], [], []
        
#         if labels is None:
#             labels = [[] for i in range(len(edu_lst))]
#         ans_word_len = 3
#         answer_words = []
        
#         for deps, label in tqdm(zip(edu_lst, labels)):
#             seq_len = len(deps)

#             word_lst = [] 
#             rel_attr = {'input_ids':torch.Tensor(), 'token_type_ids':torch.Tensor(), 'attention_mask':torch.Tensor()}
#             head_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)  # same as root index is 0, constrainting by mask 
#             rel_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
#             mask_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
#             ans_tokens = torch.zeros((ans_word_len, len(self.cfg.tokenizer)))
#             for i, dep in enumerate(deps):
#                 if i == seq_len or i + 1== self.cfg.max_length:
#                     break

#                 word_lst.append(dep.word)

#                 if dep.head in ['_', '-1'] or int(dep.head) + 1 >= self.cfg.max_length:
#                     head_tokens[i+1] = 0
#                     mask_tokens[i+1] = 0
#                 else:
#                     head_tokens[i+1] = int(dep.head)
#                     mask_tokens[i+1] = 1

#                 if self.train:
#                     rel_tokens[i+1] = rel2id[dep.rel]
#                 else:
#                     rel_tokens[i+1] = rel2id.get(dep.rel, rel2id['adjct'])
            
#             word_lst = self.prompt.split(' ') + ['[SEP]'] + word_lst
            
#             tokenized = self.cfg.tokenizer.encode_plus(word_lst, 
#                                                       padding='max_length', 
#                                                       truncation=True,
#                                                       max_length=self.cfg.max_length, 
#                                                       return_offsets_mapping=True, 
#                                                       return_tensors='pt',
#                                                       is_split_into_words=True)
#             inputs.append({"input_ids": tokenized['input_ids'][0],
#                           "token_type_ids": tokenized['token_type_ids'][0],
#                            "attention_mask": tokenized['attention_mask'][0]
#                           })
            
#             if ~self.inference:
#                 if len(label) == 0:
#                     ans_tokens[0:ans_word_len, self.cfg.tokenizer.none_token_ids] = 1
#                 else: 
#                     for l in label[0]:  # first label
#                         label_id = self.cfg.tokenizer(l)['input_ids'][1:-1]
#                         ans_tokens[torch.arange(len(label_id)), label_id] = 1
#                         if len(label_id) < ans_word_len:
#                             ans_tokens[torch.arange(len(label_id), ans_word_len), self.cfg.tokenizer.eos_token_id] = 1  # padding
            
#             # ans_tokens /= ans_tokens.sum(-1).unsqueeze(-1).expand(-1, len(self.cfg.tokenizer))
#                 answer_words.append(ans_tokens)

#             heads.append(head_tokens)
#             rels.append(rel_tokens)
#             masks.append(mask_tokens)
        
#         if self.inference:
#             return inputs, heads, rels, masks
#         return inputs, heads, rels, answer_words, masks

#     def __getitem__(self, idx):
#         return self.inputs[idx], self.heads[idx], self.rels[idx], self.labels[idx], self.masks[idx]
    
#     def __len__(self):
#         return len(self.rels)

In [14]:
class EduDataset(IterableDataset):
    def __init__(self, cfg, train, prompt, edu_lst, labels):
        self.train = train
        self.cfg = cfg
        self.prompt = prompt
        self.edu_lst = edu_lst
        self.labels = labels
        
    def read_data(self, edu_lst, labels):
        inputs, offsets = [], []
        tags, heads, rels, masks = [], [], [], []
        
        ans_word_len = 3
        answer_words = []
        
        for deps, label in zip(edu_lst, labels):
            seq_len = len(deps)

            word_lst = [] 
            rel_attr = {'input_ids':torch.Tensor(), 'token_type_ids':torch.Tensor(), 'attention_mask':torch.Tensor()}
            head_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)  # same as root index is 0, constrainting by mask 
            rel_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
            mask_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
            ans_tokens = torch.zeros((ans_word_len, len(self.cfg.tokenizer)))
            for i, dep in enumerate(deps):
                if i == seq_len or i + 1== self.cfg.max_length:
                    break

                word_lst.append(dep.word)

                if dep.head in ['_', '-1'] or int(dep.head) + 1 >= self.cfg.max_length:
                    head_tokens[i+1] = 0
                    mask_tokens[i+1] = 0
                else:
                    head_tokens[i+1] = int(dep.head)
                    mask_tokens[i+1] = 1

                if self.train:
                    rel_tokens[i+1] = rel2id[dep.rel]
                else:
                    rel_tokens[i+1] = rel2id.get(dep.rel, rel2id['adjct'])

            word_lst = self.prompt.split(' ') + ['[SEP]'] + word_lst
            
            tokenized = self.cfg.tokenizer.encode_plus(word_lst, 
                                              padding='max_length', 
                                              truncation=True,
                                              max_length=self.cfg.max_length, 
                                              return_offsets_mapping=False, 
                                              return_tensors='pt',
                                              is_split_into_words=True)
            tokenized = {"input_ids": tokenized['input_ids'][0],
                          "token_type_ids": tokenized['token_type_ids'][0],
                           "attention_mask": tokenized['attention_mask'][0]
                          }
            
            if len(label) == 0:
                ans_tokens[0:ans_word_len, self.cfg.tokenizer.none_token_ids] = 1
            else: 
                # label_id = self.cfg.tokenizer(l)['input_ids'][1:-1]
                label_id = label2id[label[0]]  # first label
                if len(label_id) > ans_word_len:
                    label_id = label_id[:ans_word_len]
                ans_tokens[torch.arange(len(label_id)), label_id] = 1
                if len(label_id) < ans_word_len:
                    ans_tokens[torch.arange(len(label_id), ans_word_len), self.cfg.tokenizer.eos_token_id] = 1  # padding
            
            answer_words.append(ans_tokens)

            heads.append(head_tokens)
            rels.append(rel_tokens)
            masks.append(mask_tokens)
                    
            yield tokenized, head_tokens, rel_tokens, ans_tokens, mask_tokens

    def __iter__(self):
        return self.read_data(self.edu_lst, self.labels)

In [15]:
prompt = '能 表现 篇章 语义 信号 的 词 是 ： [MASK] [MASK] [MASK]'
prompt_tokenized = CFG.tokenizer.encode_plus(prompt.split(' '), is_split_into_words=True, return_tensors='pt')
print(prompt_tokenized['input_ids'])

masked_idx = [x[1].item() for x in (prompt_tokenized['input_ids'] == CFG.tokenizer.mask_token_id).nonzero()]
CFG.masked_idx = masked_idx
print(masked_idx)

tensor([[ 101, 5543, 6134, 4385, 5063, 4995, 6427,  721,  928, 1384, 4638, 6404,
         3221, 8038,  103,  103,  103,  102]])
[14, 15, 16]


In [16]:
if CFG.debug:
    dataset = EduDataset(CFG, True, prompt, edu_lst[:3000], labels[:3000])
else:
    dataset = EduDataset(CFG, True, prompt, edu_lst, labels=labels)

In [17]:
# torch.save(dataset, '../data_saved/mlm/diag_codt/dataset.pt')

In [18]:
# dataset = torch.load('/root/autodl-tmp/data/sdiag_codt/dataset.pt')

In [19]:
# logit = torch.randn(2, len(CFG.tokenizer))
# gt = dataset[0][-2]

# print(logit.shape)
# print(gt.shape)

# F.cross_entropy(logit, gt)

# Model

In [20]:
class PromptModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
    
        self.encoder = AutoModel.from_pretrained(cfg.plm)
        self.encoder.resize_token_embeddings(len(cfg.tokenizer))

        self.mlm_head = nn.Linear(self.encoder.config.hidden_size, len(cfg.tokenizer))

        self.dropout = nn.Dropout(cfg.dropout)

    def forward(self, inputs, labels=None):
        feats, *_ = self.encoder(**inputs, return_dict=False)   # batch_size, seq_len (tokenized), plm_hidden_size
        feats = self.dropout(feats)

        masked_feats = feats[:, self.cfg.masked_idx, :]   # batch_size, masked_len, plm_hidden_size

        logit = self.mlm_head(masked_feats)  # batch_size, masked_len, vocab_size

        if labels is not None:
            loss = F.cross_entropy(logit, labels)  # labels: [batch_size, masked_len, vocab_size]
            return logit, loss
        return logit

In [21]:
model = PromptModel(CFG)

model.load_state_dict(torch.load('../results/prompt_model_diag.pt'))

Some weights of the model checkpoint at hfl/chinese-electra-180g-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

# Inference

In [22]:
data_iter = DataLoader(dataset, batch_size=CFG.batch_size)

In [23]:
# model = model.cuda()

# labels_whole, masks_whole = torch.Tensor(), torch.Tensor()
# logits = torch.Tensor()
# for batch in tqdm(data_iter):
#     inputs, heads, rels, labels, masks = batch

#     if CFG.cuda and torch.cuda.is_available():
#         inputs_cuda = {}
#         for key, value in inputs.items():
#             inputs_cuda[key] = value.squeeze(1).cuda()
#         inputs = inputs_cuda
        
#         heads, rels, labels, masks = to_cuda(data=(heads, rels, labels, masks))
    
#     with torch.no_grad():
#         logit = model(inputs, labels=None)
        
#     logits = torch.cat([logits, logit.cpu()], dim=0)
#     labels_whole = torch.cat([labels_whole, labels.cpu()], dim=0)
#     masks_whole = torch.cat([masks_whole, masks.cpu()], dim=0)

In [24]:
# loss = F.cross_entropy(logits.view(-1, labels_whole.size()[-1]), labels_whole.view(-1, labels_whole.size()[-1]), reduction='none')
# loss = loss.view(labels_whole.size()[0], labels_whole.size()[1])
# print(loss.shape)

# loss = loss.sum(1).mean()
# print(loss)

In [25]:
signal2slices = {}
id2pred = []

tmp = -1
for i, (signal, id) in enumerate(signal_dct.items()):
    # if tmp != -1 and id != tmp:
    #     signal2slices[tmp].append(i)
    if len(signal2slices.get(id, [])) == 0:
        signal2slices[id] = [i]
        id2pred.append(id)
        tmp = id
    else:
        signal2slices[id].append(i)
# signal2slices[id].append(i)
signal2slices[27] = [len(signal_dct.items())]
id2pred.append(27)

print(signal2slices)
print(id2pred)

{21: [0, 1, 2, 3, 4, 5, 7], 39: [6, 76, 77, 78, 79], 25: [8, 9, 10, 11, 12, 13, 14], 23: [15, 16, 17, 18, 19, 20, 21], 26: [22, 23, 24, 25, 26, 27, 28, 29, 30], 31: [31, 32, 33], 22: [34, 35, 36], 34: [37, 38, 39, 40, 41, 42, 43, 44], 28: [45, 46, 47, 48], 32: [49, 50, 51], 33: [52, 53, 54, 55, 56], 24: [57, 58], 30: [59, 60, 61, 62], 29: [63, 64, 65, 66, 67, 68, 69, 70], 36: [71, 72], 37: [73, 74, 75], 27: [80]}
[21, 39, 25, 23, 26, 31, 22, 34, 28, 32, 33, 24, 30, 29, 36, 37, 27]


In [26]:
model = model.cuda()

In [63]:
def predict(x):
    tokenized = tokenizer.encode_plus(x, 
                                      padding='max_length', 
                                      truncation=True,
                                      max_length=CFG.max_length, 
                                      return_offsets_mapping=False, 
                                      return_tensors='pt',
                                      is_split_into_words=True)

    inputs_cuda = {}
    for key, value in tokenized.items():
        if key != 'offset_mapping':
            inputs_cuda[key] = value.cuda()
    tokenized = inputs_cuda

    with torch.no_grad():
        output = model(tokenized)

    sim = (output.view(output.size()[0], -1) @ targets).cpu()[0].softmax(-1)
    # print(sim)
    # sims = torch.cat([sims, sim.cpu()], dim=0)
    
    max_probs, max_is = [0.05, 0.05], [27, 38]
    for i, (k, v) in enumerate(signal2slices.items()):
        prob = sim[v].sum() / len(v)
        # prob = sim[0][v[:2]].sum() / 2
        
        if id2pred[i] < 35 and prob > max_probs[0]:
            max_probs[0] = prob
            max_is[0] = id2pred[i]
        elif prob > max_probs[1]:
            max_probs[1] = prob
            max_is[1] = id2pred[i]
#         if prob > max_probs[1]:
#             max_probs[1] = prob
#             max_is[1] = i
            
#             if prob > max_probs[0]:
#                 max_probs[0], max_probs[1] = max_probs[1], max_probs[0]
#                 max_is[0], max_is[1] = max_is[1], max_is[0]
    
    # pred = id2pred[max_i]
    first, second = max_is[0], max_is[1]
    return first, second

In [64]:
ans_word_len = 3
targets = torch.Tensor()
for key in signal_dct.keys():
    target = torch.zeros(ans_word_len, len(CFG.tokenizer)).int()
    t = CFG.tokenizer(key)['input_ids'][1:-1]
    if len(t) > ans_word_len:
        t = t[:ans_word_len]
    target[torch.arange(len(t)), t] = 1
    if len(t) < ans_word_len:
        target[torch.arange(len(t), ans_word_len), CFG.tokenizer.eos_token_id] = 1
    targets = torch.cat([targets, target.unsqueeze(0)], dim=0)
    
target = torch.zeros(ans_word_len, len(CFG.tokenizer)).int()
target[0:ans_word_len, CFG.tokenizer.none_token_ids] = 1
targets = torch.cat([targets, target.unsqueeze(0)], dim=0)
print(targets.shape)

torch.Size([81, 3, 21132])


In [65]:
# preds = torch.Tensor()
f = open('/root/autodl-tmp/output.conll', 'w+', encoding='utf-8')

targets = targets.view(targets.size()[0], -1).T.cuda()
sims = torch.Tensor()
cnt = 0
for sample in tqdm(sample_lst):
    word_lst = []
    signal = -1
    saves = []
    for i, dep in enumerate(sample[:-1]):
        word_lst.append(dep.word)
        saves.append([dep.id, dep.word, dep.head, dep.rel])
        
        if dep.word in punct_lst and sample[i+1].word not in punct_lst:
            x = prompt.split(' ') + ['[SEP]'] + word_lst
            word_lst = []
            signal1, signal2 = predict(x)
            # preds = torch.cat([preds, torch.tensor([pred])], dim=0)
            cnt += 1
            for save in saves:
                line = f'{save[0]}\t{save[1]}\t{signal1}\t{signal2}\t_\t_\t{save[2]}\t{save[3]}\t_\t_\n'
                f.write(line)
            saves = []
    
    if i + 1 < CFG.max_length and len(sample) > 0:
        word_lst.append(sample[-1].word)
        saves.append([sample[-1].id, sample[-1].word, sample[-1].head, sample[-1].rel])
        
        x = prompt.split(' ') + ['[SEP]'] + word_lst
        signal1, signal2 = predict(x)
        # preds = torch.cat([preds, torch.tensor([pred])], dim=0)
        cnt += 1
        for save in saves:
            line = f'{save[0]}\t{save[1]}\t{signal1}\t{signal2}\t_\t_\t{save[2]}\t{save[3]}\t_\t_\n'
            f.write(line)
            
    f.write('\n')

# print(preds.shape)
print(cnt)
f.close()

  1%|          | 201/20086 [00:02<04:10, 79.46it/s]


KeyboardInterrupt: 

In [30]:
from itertools import chain

def load_codt_signal(data_file: str):
    sentence:List[Dependency] = []

    with open(data_file, 'r', encoding='utf-8') as f:
        # data example: 1	上海	_	NR	NR	_	2	nn	_	_
        for line in f.readlines():
            toks = line.split()
            if len(toks) == 0 and len(sentence) != 0:
                yield sentence
                sentence = []
            elif len(toks) == 10:
                signal = int(toks[2])
                sentence.append(signal)

In [31]:
cnt = 0
for sentence in load_codt_signal('/root/autodl-tmp/output.conll'):
    cnt += 1
print(cnt)

20086


In [32]:
f = open('/root/autodl-tmp/output.conll', 'r', encoding='utf-8')

cnt = 0
for line in f.readlines():
    if line == '\n':
        cnt += 1

print(cnt)
f.close()

20086


In [ ]:
assert False

# Case

In [35]:
prompt = '能 表现 篇章 语义 信号 的 词 是 ： [MASK] [MASK] [MASK]'

In [36]:
text = '没有 付款'

In [37]:
id2signal = list(signal_dct.keys())
print(id2signal)

['说', '表示', '看到', '显示', '知道', '认为', '希望', '指出', '如果', '假如', '的话', '若', '如', '要是', '倘若', '因为', '所以', '导致', '因此', '造成', '由于', '因而', '但是', '可是', '但', '竟', '却', '不过', '居然', '而是', '反而', '以及', '并且', '或者', '对于', '自从', '上次', '明天', '晚上', '到时候', '再', '然后', '接下来', '最后', '随后', '为了', '使', '为 的', '为 了', '通过', '必须', '点击', '对 吗', '是 吗', '对 吧', '是 吧', '对 ?', '别 的', '另外', '解释', '比如', '例如', '是 这样', '理想', '真 棒', '太 棒', '真差', '太 差', '不 行', '扯皮', '这么 麻烦', '情况', '为什么', '吗', '?', '什么', '请', '麻烦', '让', '咨询']


In [67]:
word_lst = text.split(' ')
x = prompt.split(' ') + ['[SEP]'] + word_lst
print(x)

['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '没有', '付款']


In [68]:
signal1, signal2 = predict(x)

In [70]:
signal1, signal2

(27, 38)

In [40]:
tokenized = tokenizer.encode_plus(x, 
                                  padding='max_length', 
                                  truncation=True,
                                  max_length=CFG.max_length, 
                                  return_offsets_mapping=True, 
                                  return_tensors='pt',
                                  is_split_into_words=True)
inputs_cuda = {}
for key, value in tokenized.items():
    if key != 'offset_mapping':
        inputs_cuda[key] = value.cuda()
tokenized = inputs_cuda

In [73]:
signal_dct[labels[0][0]]

37

In [91]:
targets = targets.view(targets.size()[0], -1).T.cuda()
sims = torch.Tensor()
cnt = 0
for sample, label in tqdm(zip(sample_lst, labels)):
    word_lst = []
    signal = -1
    saves = []
    for i, dep in enumerate(sample[:-1]):
        word_lst.append(dep.word)
        saves.append([dep.id, dep.word, dep.head, dep.rel])
        
        if dep.word in punct_lst and sample[i+1].word not in punct_lst:
            x = prompt.split(' ') + ['[SEP]'] + word_lst
            # for w in ['但是', '但', '可是', '却']:
            for w in ['如果', '的话', '假如']:
                if w in x:
                    x.remove(w)
                    print(x)
                    signal1, signal2 = predict(x)
                    print(signal1)
            cnt += 1
            saves, word_lst = [], []
    
    if i + 1 < CFG.max_length and len(sample) > 0:
        word_lst.append(sample[-1].word)
        saves.append([sample[-1].id, sample[-1].word, sample[-1].head, sample[-1].rel])
        
        x = prompt.split(' ') + ['[SEP]'] + word_lst
        signal1, signal2 = predict(x)
        
        cnt += 1

117it [00:01, 84.33it/s]

['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '亲亲', '您', '对', '我们', '的', '产品', '了解', '比较', '清楚', '，']
27


243it [00:02, 84.35it/s]

['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '您', '需要', '内存卡', '，']
27


558it [00:06, 81.04it/s]

...

['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '问题', '解决', '好', '了', '，']
27
['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '您', '还', '有', '问题', '可以', '随时', '提问', '哦', '，']
25


17652it [03:09, 92.47it/s]

['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '是', '零钱', '支付', '的', '，']
27
['能', '表现', '篇章', '语义', '信号', '的', '词', '是', '：', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '银行卡', '，']
27


17670it [03:09, 93.10it/s]


KeyboardInterrupt: 